<a href="https://colab.research.google.com/github/YasirISkhan/Fine-tuning-Foundation-Model-to-Generate-Accurate-Recipe/blob/main/Coding_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fatima Fellowship Coding Challenge: Finetune a Generative AI Model

I want to finetune a Generative AI Model that takes ingredients as input and generate an accurate recipe specific to the Indian cuisine.

Finetuning a generation model to be able to generate accurate recipes about a particular dish specific to your cuisine.

# 1. Building Dataset

There is an existing dataset available on Kaggle which I used to extract some of the samples. I used this particular dataset because it fulfills my requirements of including Indian cuisine for recipe generation. The dataset contained cuisines from various cultures, but I extracted only the samples specific to Indian cuisine.

I have extracted more than 600 samples from the dataset. I also performed comprehensive data analysis to check for incorrect, incomprehensible, irrelavant data. I removed the irrelevant features, duplicate values, and samples that were in languages other than English. I imputated the null values as there were less number of fields. Similarly, I cleaned the text from extra spaces. In order to make sure diversity of the dataset, I included course and diet fields which make the data more diverse.

The address of the complete dataset is available at: https://www.kaggle.com/datasets/rumitpathare/indian-recipes

In [1]:
# My CODE snippets TO BUILD THE DATASET

# Importing all essential Python Libraries
import numpy as np
import pandas as pd

df = pd.read_csv('Recipe_Generation_Dataset.csv')

# Data Preprocessing
#Filling Null values
df['course'].fillna(df['course'].mode()[0], inplace=True)
df['diet'].fillna(df['diet'].mode()[0], inplace=True)

# Normalizing values
df['ingredients_name'] = df['ingredients_name'].str.lower()

# Example text cleaning for instructions (removing extra whitespace)
df['instructions'] = df['instructions'].str.strip()

FileNotFoundError: [Errno 2] No such file or directory: 'Recipe_Generation_Dataset.csv'

**Link to the Preprocessed dataset on Hugging Face Hub:** [https://huggingface.co/datasets/Yasir-khan/Recipe_Generation_Dataset]


# 2. Finetune a Foundation Model


In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

from transformers import GPT2TokenizerFast, GPT2LMHeadModel, AutoTokenizer
from transformers import Trainer, TrainingArguments
from tqdm.auto import tqdm

In [2]:
!pip3 install datasets
import datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

AttributeError: module 'pyarrow.lib' has no attribute 'ListViewType'

In [3]:
model_name = 'gpt2'

In [4]:
pip install tqdm

In [5]:
model_save_path = '/content/drive/MyDrive/CSERA'

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
#load a pre-trained GPT-2 tokenizer and model while customizing some special tokens
tokenizer = GPT2TokenizerFast.from_pretrained(model_name,
                                              bos_token='<|startoftext|>',
                                              eos_token='<|endoftext|>',
                                              unk_token='<|unknown|>',
                                              pad_token='<|pad|>'
                                             )
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50260, 768)

In [9]:
tokenizer.save_pretrained(model_save_path)

('/content/drive/MyDrive/CSERA/tokenizer_config.json',
 '/content/drive/MyDrive/CSERA/special_tokens_map.json',
 '/content/drive/MyDrive/CSERA/vocab.json',
 '/content/drive/MyDrive/CSERA/merges.txt',
 '/content/drive/MyDrive/CSERA/added_tokens.json',
 '/content/drive/MyDrive/CSERA/tokenizer.json')

In [10]:
tokenizer.convert_tokens_to_ids(['<|pad|>'])

[50259]

In [11]:
#  take a text prompt as input and use a pre-trained GPT-2 model to generate and print a sequence of text based on that prompt
def generate(prompt):
    inputs = tokenizer.encode_plus(prompt, return_tensors='pt')
    output = model.generate(**inputs,max_length=256,do_sample=True,pad_token_id=50259)
    print(tokenizer.decode(output[0]))

In [12]:
tokenizer.special_tokens_map

{'bos_token': '<|startoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|unknown|>',
 'pad_token': '<|pad|>'}

In [13]:
tokenizer.convert_tokens_to_ids(['<|startoftext|>'],)

[50257]

In [14]:
# Loading dataset
from datasets import load_dataset

ds = load_dataset("Yasir-khan/Recipe_Generation_Dataset")
def print_recipe(idx):
    print(f"{ds['ingredients_name'][idx]}\n\n{ds['instructions'][idx]}")

Generating train split:   0%|          | 0/1065 [00:00<?, ? examples/s]

In [15]:
def form_string(ingredient_name,instructions):
    s = f"<|startoftext|>Ingredients:\n{ingredient_name.strip()}\n\nInstructions:\n{instructions.strip()}<|endoftext|>"
    return s

In [16]:
#defining a function that processes a dataset of recipes by combining the ingredients
#and instructions into a single string for each example
def apply_form_string(example):
    return {"combined": form_string(example['ingredients_name'], example['instructions'])}

# Use the map function to apply the function to the dataset
mapped_ds = ds.map(apply_form_string)

# Extract the combined data as a list
data = mapped_ds['train']['combined']

Map:   0%|          | 0/1065 [00:00<?, ? examples/s]

In [17]:
train_size = 0.85
train_len = int(train_size * len(data))
train_data = data[:train_len]
val_data = data[train_len:]

In [18]:
from tqdm import tqdm
class RecipeDataset:
    def __init__(self,data):
        self.data = data
        self.input_ids = []
        self.attn_masks = []

        for item in tqdm(data):
            encodings = tokenizer.encode_plus(item,
                                              truncation=True,
                                              padding='max_length',
                                              max_length=1024,
                                              return_tensors='pt'
                                             )
            self.input_ids.append(torch.squeeze(encodings['input_ids'],0))
            self.attn_masks.append(torch.squeeze(encodings['attention_mask'],0))

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [19]:
def collate_fn(batch):
    return {
        'input_ids': torch.stack([item[0] for item in batch]),
        'attention_mask': torch.stack([item[1] for item in batch]),
        'labels': torch.stack([item[0] for item in batch])
    }

In [20]:
train_ds = RecipeDataset(train_data)
val_ds = RecipeDataset(val_data)

100%|██████████| 160/160 [00:00<00:00, 481.96it/s]


In [ ]:
pip install transformers[torch]


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
pip show transformers accelerate

Name: transformers
Version: 4.42.4
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
---
Name: accelerate
Version: 0.32.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [21]:
from transformers import TrainingArguments

model_save_path = './model_save_path'  # Ensure this is defined

args = TrainingArguments(
    output_dir=model_save_path,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    report_to='none',
    num_train_epochs=3,
    save_strategy='no'
)


In [22]:
args = TrainingArguments(output_dir=model_save_path,
                         per_device_train_batch_size=2,
                         per_device_eval_batch_size=2,
                         gradient_accumulation_steps=2,
                         report_to='none',
                         num_train_epochs=3,
                         save_strategy='no'
                        )


In [23]:
optim = torch.optim.AdamW(model.parameters(),lr=5e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim,20,eta_min=1e-7)

In [24]:
trainer = Trainer(model,
                  args,
                  train_dataset=train_ds,
                  eval_dataset=val_ds,
                  data_collator=collate_fn,
                  optimizers=(optim,scheduler)
                 )

In [25]:
trainer = Trainer(model,
                  args,
                  train_dataset=train_ds,
                  eval_dataset=val_ds,
                  data_collator=collate_fn,
                  optimizers=(optim,scheduler)
                 )

In [26]:
trainer.train()

Step,Training Loss
500,0.885600


TrainOutput(global_step=678, training_loss=0.8215078505794559, metrics={'train_runtime': 911.8389, 'train_samples_per_second': 2.977, 'train_steps_per_second': 0.744, 'total_flos': 1416202813440000.0, 'train_loss': 0.8215078505794559, 'epoch': 2.993377483443709})

In [27]:
trainer.save_model()


In [28]:
from transformers import pipeline


In [29]:
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer # Import necessary classes

model_save_path = '/content/model_save_path'  # Path where model files and config.json are located
tokenizer_path = '/content/drive/MyDrive/CSERA'

model = AutoModelForCausalLM.from_pretrained(model_save_path)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

pl = pipeline(task='text-generation', model=model, tokenizer=tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [30]:
def create_prompt(ingredients):
    ingredients = ','.join([x.strip().lower() for x in ingredients.split(',')])
    ingredients = ingredients.strip().replace(',','\n')
    s = f"<|startoftext|>Ingredients:\n{ingredients}\n"
    return s

In [31]:
ingredients = ['Rice,Potatoes,Tomatoes,Spinach,red bell peppers','chicken,tomatoes,aloo,jeera,curry powder']

In [32]:
ingredients = ['Rice']

In [33]:
for ing in ingredients:
    prompt = create_prompt(ing)
    print(pl(prompt,
         max_new_tokens=512,
         penalty_alpha=0.6,
         top_k=4,
         pad_token_id=50259
        )[0]['generated_text'])

<|startoftext|>Ingredients:
rice
semiya (dahi / yogurt), cumin seeds (jeera), green chillies, turmeric powder (haldi), red chilli powder, garam masala powder, salt, coriander powder (dhania)

Instructions:
To begin making the Rice Recipe, heat oil in the same kadai and add the chopped rice. Cook for 3 to 4 whistles until the rice is cooked through. Once the rice is cooked, add in the cumin seeds and green chillies. Serve Rice Recipe along with Dhaniya Chutney Recipe or Paneer Chutney Recipe for a simple breakfast along with Pudina Paratha.


In [34]:
ingredients = ['Chiken']

In [35]:
for ing in ingredients:
    prompt = create_prompt(ing)
    print(pl(prompt,
         max_new_tokens=512,
         penalty_alpha=0.6,
         top_k=4,
         pad_token_id=50259
        )[0]['generated_text'])

<|startoftext|>Ingredients:
chiken
capsicum, onion, tomato, garlic, green bell pepper (capsicum), coriander (dhania) leaves, green chillies, salt

Instructions:
To begin making the Capsicum Onion Raita, wash the capsicum and cut it into equal portions.Heat some oil in a pan and add the onion and cook till the onions turn golden brown.Add the tomatoes and cook till they turn soft. Add the capsicum and cook for another 2 to 3 minutes.Add the tomatoes and cook till tomatoes turn mushy.Add salt, green chillies and green coriander leaves to this.Add the capsicum and cook till tomatoes become mushy.Add the tomatoes and cook for another 2 to 3 minutes. Add the capsicum and cook till tomatoes turn mushy and turn golden brown.Add the remaining capsicum and cook till tomatoes turn mushy and turn brown.Serve Capsicum Onion Raita along with Tomato Onion Raita Recipe for a weeknight dinner along with some Phulkas and Tomato Onion Raita Recipe for your everyday meal.


In [36]:
ingredients = ['meet']
for ing in ingredients:
    prompt = create_prompt(ing)
    print(pl(prompt,
         max_new_tokens=512,
         penalty_alpha=0.6,
         top_k=4,
         pad_token_id=50259
        )[0]['generated_text'])

<|startoftext|>Ingredients:
meet
semolina (semolina), mustard seeds (rai/ kadugu), green chillies, coriander (dhania) leaves, lemon juice, oil, salt, mustard seeds (rai/ kadugu), green chillies, coriander leaves, lemon juice, oil

Instructions:
To prepare met methi, first soak methi in water for about 30 minutes. Drain the water from the methi.Heat oil in a kadai and add mustard seeds.Add lemon juice and saute for a minute.Add methi and saute till it becomes a light brown colour.Add mustard seeds to methi mixture. Add lemon juice and saute for a minute.Add salt to methi mixture. Mix well. Add lemon juice, saute for 5-6 seconds, and serve.Serve methi along with Pudina Chai Recipe and Pudina Chai Salad Recipe for your lunch. You can also serve it as a snack along with Paneer Paneer Recipe and Masoodi Chutney Recipe for your everyday snacks.


# Generating Samples from Base model of GPT2

In [37]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
model_Name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_Name)
tokenizer = GPT2Tokenizer.from_pretrained(model_Name)


In [38]:
import torch

# Defining the prompt that was given to the fine-tuned model
prompt = "Creat a recipe using the following Ingredients: meet, rice, ghee, lemon juice and salt"

# Encode the prompt into tokens
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Generating text
output = model.generate(input_ids, max_length=200, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)

# Decoding the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Creat a recipe using the following Ingredients: meet, rice, ghee, lemon juice and salt

1/2 cup water
.
, or 1/4 cup sugar
:
- 1 cup of water, divided
 (or 1 tablespoon of sugar)
/ 1 teaspoon salt, to taste
*If you use a rice cooker, you can use 1 1⁄2 cups of rice per 1 pound of body weight. If you cook rice in a large pot, use 2 cups per pound. You can also use more water if you want to use less water.


Prepare the rice. In a medium bowl, combine the water and sugar. Add the salt and lemon. Stir well. Pour the mixture into a small bowl. Cover the bowl with plastic wrap and let sit for 10 minutes. Remove from the heat and allow to cool completely. Drain the excess water from your rice and place it in the refrigerator. Let it sit in your refrigerator for at least 2


In [ ]:
import torch

# Defining the prompt that was given to the fine-tuned model
prompt = "Make a recipe using the following Ingredients: Rice,Potatoes,Tomatoes,Spinach,red bell peppers','chicken,tomatoes,aloo,jeera,curry powder'"

# Encode the prompt into tokens
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Generating text
output = model.generate(input_ids, max_length=200, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)

# Decoding the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Make a recipe using the following Ingredients: Rice,Potatoes,Tomatoes,Spinach,red bell peppers','chicken,tomatoes,aloo,jeera,curry powder'

1/2 cup of water
.
,
:
-1 cup water,1 tablespoon of salt
 (or 1 teaspoon of pepper)
/
 and 1/4 cup chopped onion
(optional) Instructions Preheat oven to 350 degrees F. In a large bowl, combine the water and salt. Add the onion and cook for about 5 minutes. Remove from heat and add the chopped onions. Cook for another 5-10 minutes, until the onions are soft and translucent. Stir in the chicken and vegetables. Serve immediately. Recipe Notes: If you are using a rice cooker, you can use a small pot of boiling water. If using an oven, use the same amount of cooking water as the rice. You can also use your own water for the cooking. Nutrition


# Write Up:
First of all, I downloaded a recently developed recipe dataset from Kaggle platform. Having manually preprocessed the dataset and extracted the relevant features, I uploaded the cleaned dataset to the Hugging Face. Next, I selected pre-trained GPT-2 as a foundation model for my custom dataset.

Due to limited computational resources, I am unable to finetune a model from scratch. Therefore, I utilized pre-trained model. I fed data to that pre-trained model. Next, I created embeddings from the data. Moreover, I passed those embeddings to the model to generate my desired text. Resultantly, the model generates recipe based on the embeddings that are created from the data.


For fine-tuning, I utilized transformers library because it provides access to a wide variety of pre-trained models, including GPT-2. Similarly, the transformers library offers a high-level API that simplifies the process of loading models and tokenizers, performing tokenization, and generating text. Last but not least, the transformers library includes powerful tokenization utilities, such as GPT2TokenizerFast, which are optimized for performance and support a wide range of languages and special tokens.

In terms of comparison with base model of GPT-2, one can clearly see the difference between both based model and pre-trained model. Although the base model generates some level of recipe, the fine-tuned model generates recipes to the maximum accurate level. Again, the limited computational power limited my ability to further fine-tune the model and generate best results.

Link to the model on Hugging Face Hub: https://huggingface.co/Yasir-khan/Recipe-Using-GPT2